In [8]:
import config

import logging
import pandas as pd

logger = logging.getLogger(__name__)

2023-04-20 17:56:56,636 - INFO     | config     | Loading environment variables


In [10]:
import glob
import pickle

files = glob.glob('embeddings/*_test.pkl')
embeddigs_data = []

for file in files:
    logger.info(f'Loading {file}')
    with open(file, 'rb') as fp:
        dict = pickle.load(fp)
        embeddigs_data.append(dict)

2023-04-20 17:57:08,875 - INFO     | __main__   | Loading embeddings/glove_embeddings_test.pkl
2023-04-20 17:57:08,876 - INFO     | __main__   | Loading embeddings/mpnet_embeddings_test.pkl
2023-04-20 17:57:08,878 - INFO     | __main__   | Loading embeddings/distil_embeddings_test.pkl
2023-04-20 17:57:08,879 - INFO     | __main__   | Loading embeddings/wiki_embeddings_test.pkl
2023-04-20 17:57:08,880 - INFO     | __main__   | Loading embeddings/w2v_embeddings_test.pkl


# Clustering

## HDBSCAN

In [ ]:
import hdbscan